In [1]:
!pip -q install transformers>=4.38.0 trl>=0.8.0 peft>=0.9.0 bitsandbytes evaluate

from datasets import Dataset
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments, AutoTokenizer
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import evaluate
from tqdm.auto import tqdm
from tqdm.auto import tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.

2025-11-09 16:52:28.567466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762707148.961033      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762707149.097210      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
checkpoint = 'Vikhrmodels/Vikhr-Qwen-2.5-1.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

def template_processing(question, answer=None):
    if answer is None:
        return tokenizer.apply_chat_template(
            [{'role': 'user', 'content': f'Переведи это в json формат:\n\n{question}'}],
            tokenize=False,
            add_generation_promt=True
        )
    else:
        return tokenizer.apply_chat_template(
            [{'role': 'user', 'content': f'Переведи это в json формат:\n\n{question}'},
             {'role': 'assistant', 'content': answer}],
            tokenize=False
        )

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [6]:
import json
import random

# Списки для генерации
names = [
    "Анна", "Мария", "Елена", "Ольга", "Наталья", "Татьяна", "Ирина", "Светлана", 
    "Екатерина", "Юлия", "Людмила", "Галина", "Валентина", "Нина", "Зоя", "Лариса",
    "Александр", "Сергей", "Владимир", "Алексей", "Дмитрий", "Андрей", "Николай",
    "Иван", "Михаил", "Павел", "Петр", "Константин", "Виктор", "Борис", "Григорий",
    "Евгений", "Олег", "Игорь", "Роман", "Денис", "Максим", "Артем", "Кирилл",
    "Вера", "Надежда", "Любовь", "София", "Дарья", "Ксения", "Марина", "Алина",
    "Полина", "Виктория", "Анастасия", "Евгения", "Арина", "Яна", "Карина",
    "Илья", "Егор", "Тимур", "Руслан", "Антон", "Вадим", "Никита", "Данил",
    "Станислав", "Леонид", "Геннадий", "Валерий", "Федор", "Степан", "Семен",
    "Инна", "Раиса", "Клара", "Роза", "Майя", "Эльвира", "Жанна", "Лилия",
    "Тамара", "Алла", "Римма", "Зинаида", "Лидия", "Маргарита", "Ангелина",
    "Кристина", "Диана", "Милана", "Камила", "Элина", "Регина", "Амина",
    "Ярослав", "Глеб", "Тарас", "Родион", "Арсений", "Платон", "Мирон",
    "Филипп", "Эдуард", "Альберт", "Артур", "Герман", "Марк", "Лев"
]

products = {
    "электроника": ["телефон", "ноутбук", "планшет", "наушники", "колонки", "монитор", 
                    "клавиатура", "мышь", "веб-камера", "микрофон", "принтер", "сканер",
                    "роутер", "флешка", "жесткий диск", "видеокарта", "процессор", "блок питания",
                    "телевизор", "фотоаппарат", "видеокамера", "смарт-часы", "фитнес-браслет"],
    
    "продукты": ["хлеб", "молоко", "яйца", "сыр", "творог", "йогурт", "кефир", "сметана",
                 "масло", "мука", "сахар", "соль", "макароны", "рис", "гречка", "овсянка",
                 "яблоки", "бананы", "апельсины", "мандарины", "лимоны", "груши", "виноград",
                 "помидоры", "огурцы", "картофель", "морковь", "лук", "чеснок", "капуста",
                 "курица", "говядина", "свинина", "рыба", "колбаса", "сосиски", "пельмени"],
    
    "одежда": ["футболка", "рубашка", "джинсы", "брюки", "юбка", "платье", "куртка",
               "пальто", "свитер", "кофта", "майка", "шорты", "носки", "трусы", "бюстгальтер",
               "колготки", "галстук", "ремень", "шапка", "шарф", "перчатки", "кепка",
               "кроссовки", "туфли", "ботинки", "сапоги", "босоножки", "тапочки"],
    
    "канцтовары": ["ручка", "карандаш", "ластик", "линейка", "тетрадь", "блокнот",
                   "альбом", "краски", "кисти", "фломастеры", "маркер", "корректор",
                   "скрепки", "кнопки", "скотч", "клей", "ножницы", "степлер", "дырокол",
                   "папка", "файл", "конверт", "бумага", "картон", "пенал", "рюкзак"],
    
    "посуда": ["тарелка", "чашка", "стакан", "кружка", "вилка", "ложка", "нож",
               "кастрюля", "сковорода", "чайник", "заварочный чайник", "сахарница",
               "салатник", "блюдо", "поднос", "графин", "бокал", "рюмка", "ваза",
               "контейнер", "банка", "бутылка", "термос", "фляга"],
    
    "мебель": ["стол", "стул", "кресло", "диван", "кровать", "шкаф", "комод",
               "тумбочка", "полка", "стеллаж", "вешалка", "зеркало", "пуфик",
               "табурет", "скамейка", "кушетка", "софа", "буфет", "сервант"],
    
    "косметика": ["шампунь", "гель для душа", "мыло", "крем", "лосьон", "тоник",
                  "маска для лица", "скраб", "пилинг", "сыворотка", "помада", "тушь",
                  "тени", "румяна", "пудра", "тональный крем", "консилер", "бронзер",
                  "лак для ногтей", "духи", "дезодорант", "зубная паста", "зубная щетка"],
    
    "бытовая химия": ["стиральный порошок", "кондиционер для белья", "отбеливатель",
                      "средство для посуды", "средство для стекол", "средство для пола",
                      "средство для ванной", "средство для унитаза", "освежитель воздуха",
                      "салфетки", "губка", "тряпка", "перчатки", "мешки для мусора"],
    
    "игрушки": ["кукла", "машинка", "конструктор", "пазл", "мяч", "скакалка",
                "обруч", "настольная игра", "карты", "домино", "шахматы", "шашки",
                "мягкая игрушка", "робот", "самолет", "вертолет", "поезд", "корабль"],
    
    "книги": ["учебник", "роман", "детектив", "фантастика", "поэзия", "энциклопедия",
              "словарь", "журнал", "газета", "комикс", "раскраска", "кроссворды",
              "путеводитель", "кулинарная книга", "самоучитель"],
    
    "цветы": ["роза", "тюльпан", "хризантема", "гвоздика", "лилия", "орхидея",
              "пион", "ромашка", "герань", "фиалка", "кактус", "алоэ", "фикус",
              "драцена", "пальма", "папоротник", "суккулент"],
    
    "инструменты": ["молоток", "отвертка", "пила", "дрель", "шуруповерт", "лобзик",
                    "напильник", "наждачная бумага", "рулетка", "уровень", "ключ",
                    "плоскогубцы", "кусачки", "паяльник", "клещи", "топор", "лопата",
                    "грабли", "тяпка", "секатор", "газонокосилка"],
    
    "спорттовары": ["гантели", "штанга", "коврик для йоги", "скакалка", "эспандер",
                    "мяч для фитнеса", "обруч", "ракетка", "воланчик", "коньки",
                    "лыжи", "сноуборд", "велосипед", "самокат", "ролики", "скейтборд"],
    
    "аксессуары": ["сумка", "рюкзак", "кошелек", "портмоне", "визитница", "чехол",
                   "браслет", "цепочка", "кольцо", "серьги", "брошь", "заколка",
                   "резинка для волос", "ободок", "очки", "часы", "зонт"],
    
    "автотовары": ["шины", "диски", "аккумулятор", "масло", "антифриз", "омывайка",
                   "свечи", "фильтр", "тормозные колодки", "амортизатор", "ремень",
                   "лампочка", "предохранитель", "коврики", "чехлы", "ароматизатор"],
    
    "еда готовая": ["пицца", "роллы", "суши", "бургер", "шаурма", "салат", "суп",
                    "паста", "лазанья", "плов", "шашлык", "котлеты", "пельмени",
                    "блины", "оладьи", "торт", "пирожное", "печенье", "конфеты"],
    
    "напитки": ["вода", "сок", "газировка", "кола", "чай", "кофе", "какао",
                "молочный коктейль", "смузи", "лимонад", "морс", "компот", "кисель",
                "пиво", "вино", "шампанское", "коньяк", "виски", "водка"],
    
    "для дома": ["подушка", "одеяло", "простыня", "наволочка", "пододеяльник",
                 "покрывало", "плед", "полотенце", "халат", "тапочки", "коврик",
                 "штора", "тюль", "жалюзи", "карниз", "люстра", "светильник",
                 "лампа", "свеча", "подсвечник", "ваза", "горшок для цветов"],
    
    "техника": ["холодильник", "стиральная машина", "посудомоечная машина",
                "микроволновка", "духовка", "плита", "вытяжка", "миксер", "блендер",
                "мясорубка", "кофемашина", "чайник", "тостер", "мультиварка",
                "пароварка", "хлебопечка", "соковыжималка", "утюг", "фен", "пылесос"],
    
    "подарки": ["открытка", "упаковка", "лента", "бант", "пакет", "коробка",
                "сувенир", "магнит", "брелок", "статуэтка", "фоторамка", "альбом",
                "свеча", "аромалампа", "диффузор", "подарочный сертификат"]
}

def get_word_form(word, quantity):
    """
    Возвращает правильную форму слова в зависимости от количества.
    Для существительных после числительных.
    """
    
    # Словарь исключений и неправильных форм
    irregular_forms = {
        # Продукты
        "яйцо": {1: "яйцо", 2: "яйца", 5: "яиц"},
        "яблоко": {1: "яблоко", 2: "яблока", 5: "яблок"},
        "молоко": {1: "литр молока", 2: "литра молока", 5: "литров молока"},
        "хлеб": {1: "буханка хлеба", 2: "буханки хлеба", 5: "буханок хлеба"},
        "батон": {1: "батон", 2: "батона", 5: "батонов"},
        "апельсин": {1: "апельсин", 2: "апельсина", 5: "апельсинов"},
        "мандарин": {1: "мандарин", 2: "мандарина", 5: "мандаринов"},
        "банан": {1: "банан", 2: "банана", 5: "бананов"},
        "помидор": {1: "помидор", 2: "помидора", 5: "помидоров"},
        "огурец": {1: "огурец", 2: "огурца", 5: "огурцов"},
        "картофель": {1: "кг картофеля", 2: "кг картофеля", 5: "кг картофеля"},
        "морковь": {1: "морковь", 2: "моркови", 5: "морковок"},
        "перец": {1: "перец", 2: "перца", 5: "перцев"},
        
        # Одежда
        "футболка": {1: "футболка", 2: "футболки", 5: "футболок"},
        "рубашка": {1: "рубашка", 2: "рубашки", 5: "рубашек"},
        "джинсы": {1: "пара джинсов", 2: "пары джинсов", 5: "пар джинсов"},
        "брюки": {1: "пара брюк", 2: "пары брюк", 5: "пар брюк"},
        "носки": {1: "пара носков", 2: "пары носков", 5: "пар носков"},
        "платье": {1: "платье", 2: "платья", 5: "платьев"},
        "юбка": {1: "юбка", 2: "юбки", 5: "юбок"},
        "куртка": {1: "куртка", 2: "куртки", 5: "курток"},
        "пальто": {1: "пальто", 2: "пальто", 5: "пальто"},
        "туфли": {1: "пара туфель", 2: "пары туфель", 5: "пар туфель"},
        "кроссовки": {1: "пара кроссовок", 2: "пары кроссовок", 5: "пар кроссовок"},
        "ботинки": {1: "пара ботинок", 2: "пары ботинок", 5: "пар ботинок"},
        "сапоги": {1: "пара сапог", 2: "пары сапог", 5: "пар сапог"},
        "перчатки": {1: "пара перчаток", 2: "пары перчаток", 5: "пар перчаток"},
        "колготки": {1: "пара колготок", 2: "пары колготок", 5: "пар колготок"},
        
        # Канцтовары
        "ручка": {1: "ручка", 2: "ручки", 5: "ручек"},
        "карандаш": {1: "карандаш", 2: "карандаша", 5: "карандашей"},
        "тетрадь": {1: "тетрадь", 2: "тетради", 5: "тетрадей"},
        "ластик": {1: "ластик", 2: "ластика", 5: "ластиков"},
        "линейка": {1: "линейка", 2: "линейки", 5: "линеек"},
        "ножницы": {1: "ножницы", 2: "ножниц", 5: "ножниц"},
        "книга": {1: "книга", 2: "книги", 5: "книг"},
        "журнал": {1: "журнал", 2: "журнала", 5: "журналов"},
        "газета": {1: "газета", 2: "газеты", 5: "газет"},
        
        # Посуда
        "тарелка": {1: "тарелка", 2: "тарелки", 5: "тарелок"},
        "чашка": {1: "чашка", 2: "чашки", 5: "чашек"},
        "стакан": {1: "стакан", 2: "стакана", 5: "стаканов"},
        "ложка": {1: "ложка", 2: "ложки", 5: "ложек"},
        "вилка": {1: "вилка", 2: "вилки", 5: "вилок"},
        "нож": {1: "нож", 2: "ножа", 5: "ножей"},
        "кастрюля": {1: "кастрюля", 2: "кастрюли", 5: "кастрюль"},
        "сковорода": {1: "сковорода", 2: "сковороды", 5: "сковород"},
        
        # Мебель
        "стул": {1: "стул", 2: "стула", 5: "стульев"},
        "стол": {1: "стол", 2: "стола", 5: "столов"},
        "кресло": {1: "кресло", 2: "кресла", 5: "кресел"},
        "диван": {1: "диван", 2: "дивана", 5: "диванов"},
        "кровать": {1: "кровать", 2: "кровати", 5: "кроватей"},
        "шкаф": {1: "шкаф", 2: "шкафа", 5: "шкафов"},
        "полка": {1: "полка", 2: "полки", 5: "полок"},
        
        # Электроника
        "телефон": {1: "телефон", 2: "телефона", 5: "телефонов"},
        "ноутбук": {1: "ноутбук", 2: "ноутбука", 5: "ноутбуков"},
        "планшет": {1: "планшет", 2: "планшета", 5: "планшетов"},
        "наушники": {1: "наушники", 2: "наушников", 5: "наушников"},
        "монитор": {1: "монитор", 2: "монитора", 5: "мониторов"},
        "мышь": {1: "мышь", 2: "мыши", 5: "мышей"},
        "клавиатура": {1: "клавиатура", 2: "клавиатуры", 5: "клавиатур"},
        
        # Цветы
        "роза": {1: "роза", 2: "розы", 5: "роз"},
        "тюльпан": {1: "тюльпан", 2: "тюльпана", 5: "тюльпанов"},
        "хризантема": {1: "хризантема", 2: "хризантемы", 5: "хризантем"},
        "гвоздика": {1: "гвоздика", 2: "гвоздики", 5: "гвоздик"},
        "лилия": {1: "лилия", 2: "лилии", 5: "лилий"},
        "пион": {1: "пион", 2: "пиона", 5: "пионов"},
        "ромашка": {1: "ромашка", 2: "ромашки", 5: "ромашек"},
        "орхидея": {1: "орхидея", 2: "орхидеи", 5: "орхидей"},
        "кактус": {1: "кактус", 2: "кактуса", 5: "кактусов"},
        
        # Еда
        "пицца": {1: "пицца", 2: "пиццы", 5: "пицц"},
        "ролл": {1: "ролл", 2: "ролла", 5: "роллов"},
        "суши": {1: "суши", 2: "суши", 5: "суши"},
        "бургер": {1: "бургер", 2: "бургера", 5: "бургеров"},
        "торт": {1: "торт", 2: "торта", 5: "тортов"},
        "пирожное": {1: "пирожное", 2: "пирожных", 5: "пирожных"},
        "печенье": {1: "печенье", 2: "печенья", 5: "печений"},
        "конфета": {1: "конфета", 2: "конфеты", 5: "конфет"},
        "пончик": {1: "пончик", 2: "пончика", 5: "пончиков"},
        "круассан": {1: "круассан", 2: "круассана", 5: "круассанов"},
        
        # Напитки
        "кофе": {1: "чашка кофе", 2: "чашки кофе", 5: "чашек кофе"},
        "чай": {1: "пакетик чая", 2: "пакетика чая", 5: "пакетиков чая"},
        "сок": {1: "литр сока", 2: "литра сока", 5: "литров сока"},
        "вода": {1: "бутылка воды", 2: "бутылки воды", 5: "бутылок воды"},
        "кола": {1: "банка колы", 2: "банки колы", 5: "банок колы"},
        
        # Инструменты
        "молоток": {1: "молоток", 2: "молотка", 5: "молотков"},
        "отвертка": {1: "отвертка", 2: "отвертки", 5: "отверток"},
        "гвоздь": {1: "гвоздь", 2: "гвоздя", 5: "гвозде��"},
        "шуруп": {1: "шуруп", 2: "шурупа", 5: "шурупов"},
        "пила": {1: "пила", 2: "пилы", 5: "пил"},
        
        # Косметика  
        "шампунь": {1: "бутылка шампуня", 2: "бутылки шампуня", 5: "бутылок шампуня"},
        "мыло": {1: "кусок мыла", 2: "куска мыла", 5: "кусков мыла"},
        "крем": {1: "тюбик крема", 2: "тюбика крема", 5: "тюбиков крема"},
        "помада": {1: "помада", 2: "помады", 5: "помад"},
        "тушь": {1: "тушь", 2: "туши", 5: "тушей"},
        
        # Разное
        "свеча": {1: "свеча", 2: "свечи", 5: "свечей"},
        "подушка": {1: "подушка", 2: "подушки", 5: "подушек"},
        "одеяло": {1: "одеяло", 2: "одеяла", 5: "одеял"},
        "полотенце": {1: "полотенце", 2: "полотенца", 5: "полотенец"},
        "простыня": {1: "простыня", 2: "простыни", 5: "простыней"},
        "ваза": {1: "ваза", 2: "вазы", 5: "ваз"},
        "зеркало": {1: "зеркало", 2: "зеркала", 5: "зеркал"},
        "лампа": {1: "лампа", 2: "лампы", 5: "ламп"},
        "батарейка": {1: "батарейка", 2: "батарейки", 5: "батареек"},
        "зарядка": {1: "зарядка", 2: "зарядки", 5: "зарядок"},
        "сумка": {1: "сумка", 2: "сумки", 5: "сумок"},
        "рюкзак": {1: "рюкзак", 2: "рюкзака", 5: "рюкзаков"},
        "кошелек": {1: "кошелек", 2: "кошелька", 5: "кошельков"},
        "часы": {1: "часы", 2: "часов", 5: "часов"},
        "браслет": {1: "браслет", 2: "браслета", 5: "браслетов"},
        "кольцо": {1: "кольцо", 2: "кольца", 5: "колец"},
        "цепочка": {1: "цепочка", 2: "цепочки", 5: "цепочек"},
        "серьги": {1: "пара серег", 2: "пары серег", 5: "пар серег"},
    }
    
    # Общие правила для слов, которых нет в словаре
    general_rules = {
        # Женский род на -а
        "ending_а": {
            "check": lambda w: w.endswith("а") and w not in irregular_forms,
            "forms": {
                1: lambda w: w,
                2: lambda w: w[:-1] + "и",
                5: lambda w: w[:-1]
            }
        },
        # Женский род на -я
        "ending_я": {
            "check": lambda w: w.endswith("я") and w not in irregular_forms,
            "forms": {
                1: lambda w: w,
                2: lambda w: w[:-1] + "и",
                5: lambda w: w[:-1] + "й" if w[-2] in "аеёиоуыэюя" else w[:-1] + "ей"
            }
        },
        # Женский род на -ь
        "ending_ь_fem": {
            "check": lambda w: w.endswith("ь") and w not in irregular_forms and w not in ["гвоздь", "шампунь", "ноготь"],
            "forms": {
                1: lambda w: w,
                2: lambda w: w[:-1] + "и",
                5: lambda w: w[:-1] + "ей"
            }
        },
        # Мужской род на согласную
        "ending_consonant": {
            "check": lambda w: w[-1] in "бвгджзклмнпрстфхцчшщ" and w not in irregular_forms,
            "forms": {
                1: lambda w: w,
                2: lambda w: w + "а",
                5: lambda w: w + "ов"
            }
        },
        # Средний род на -о
        "ending_о": {
            "check": lambda w: w.endswith("о") and w not in irregular_forms,
            "forms": {
                1: lambda w: w,
                2: lambda w: w[:-1] + "а",
                5: lambda w: w[:-1]
            }
        },
        # Средний род на -е
        "ending_е": {
            "check": lambda w: w.endswith("е") and w not in irregular_forms,
            "forms": {
                1: lambda w: w,
                2: lambda w: w[:-1] + "я",
                5: lambda w: w[:-1] + "й" if w[-2] in "ьи" else w[:-1]
            }
        }
    }
    
    # Определяем форму слова
    def get_form(word, quantity):
        # Проверяем словарь исключений
        if word in irregular_forms:
            forms = irregular_forms[word]
            if quantity == 1:
                return forms[1]
            elif quantity % 10 in [2, 3, 4] and quantity % 100 not in [12, 13, 14]:
                return forms[2]
            else:
                return forms[5]
        
        # Применяем общие правила
        for rule_name, rule in general_rules.items():
            if rule["check"](word):
                if quantity == 1:
                    return rule["forms"][1](word)
                elif quantity % 10 in [2, 3, 4] and quantity % 100 not in [12, 13, 14]:
                    return rule["forms"][2](word)
                else:
                    return rule["forms"][5](word)
        
        # Если не подходит ни одно правило, возвращаем как есть
        return word
    
    return get_form(word, quantity)

def generate_examples(num_products, count=500):
    examples = []
    actions = ["купил", "купила", "приобрел", "приобрела", "заказал", "заказала"]
    
    for _ in range(count):
        name = random.choice(names)
        action = random.choice(actions)
        
        # Корректируем окончание глагола в зависимости от пола
        if name in ["Анна", "Мария", "Елена", "Ольга", "Наталья", "Татьяна", "Ирина", 
                    "Светлана", "Екатерина", "Юлия", "Людмила", "Галина", "Валентина",
                    "Нина", "Зоя", "Лариса", "Вера", "Надежда", "Любовь", "София",
                    "Дарья", "Ксения", "Марина", "Алина", "Полина", "Виктория",
                    "Анастасия", "Евгения", "Арина", "Яна", "Карина", "Инна", "Раиса",
                    "Клара", "Роза", "Майя", "Эльвира", "Жанна", "Лилия", "Тамара",
                    "Алла", "Римма", "Зинаида", "Лидия", "Маргарита", "Ангелина",
                    "Кристина", "Диана", "Милана", "Камила", "Элина", "Регина", "Амина"]:
            if action.endswith("л"):
                action = action + "а"
        
        purchases = []
        input_parts = []
        total = 0
        
        # Выбираем категории для разнообразия
        categories = random.sample(list(products.keys()), min(num_products, len(products)))
        
        for i in range(num_products):
            category = categories[i % len(categories)]
            product = random.choice(products[category])
            quantity = random.randint(1, 20)
            
            # Генерируем цену в зависимости от категории
            if category in ["электроника", "техника", "мебель"]:
                price = random.randint(10, 100) * 1000
            elif category in ["одежда", "косметика", "книги"]:
                price = random.randint(5, 50) * 100
            elif category in ["продукты", "канцтовары", "бытовая химия"]:
                price = random.randint(1, 10) * 50 * quantity
            else:
                price = random.randint(10, 200) * 10 * quantity
            
            # Корректируем форму слова для количества
            product_form = product
            if quantity > 1:
                # Добавляем окончание для родительного падежа множественного числа
                if product.endswith('а') or product.endswith('я'):
                    product_form = product[:-1]
                elif product.endswith('о') or product.endswith('е'):
                    product_form = product[:-1]
                elif not product.endswith('ы') and not product.endswith('и'):
                    product_form = product + 'ов'
            
            # Добавляем единицы измерения для некоторых товаров
            unit = ""
            if product in ["молоко", "сок", "вода", "масло", "газировка", "кола"]:
                unit = "литр" if quantity == 1 else "литра" if quantity in [2,3,4] else "литров"
                product_form = unit + " " + product.replace('а','и') if product.endswith('а') else unit + " " + product + 'а'
            elif product in ["хлеб", "сыр", "масло", "творог", "мука", "сахар", "рис", "гречка"]:
                unit = "кг"
                product_form = unit + " " + product.replace('а','и') if product.endswith('а') else unit + " " + product + 'а'
            elif product in ["яблоки", "бананы", "апельсины", "помидоры", "огурцы", "картофель"]:
                unit = "кг"
                product_form = unit + " " + product[:-1] if product.endswith('и') else unit + " " + product + 'ов'
            
            purchases.append({
                "товар": product_form if quantity > 1 else product,
                "количество": quantity,
                "цена": price
            })
            
            total += price
            
            # Формируем часть входной строки
            if i == 0:
                input_parts.append(f"{quantity} {product_form if quantity > 1 else product} за {price} рублей")
            elif i == num_products - 1:
                input_parts.append(f"и {quantity} {product_form if quantity > 1 else product} за {price} рублей")
            else:
                input_parts.append(f", {quantity} {product_form if quantity > 1 else product} за {price} рублей")
        
        input_text = f"{name} {action} " + " ".join(input_parts)
        
        output = {
            "покупки": purchases,
            "итого": total
        }
        
        examples.append({
            "input": input_text,
            "output": json.dumps(output, ensure_ascii=False)
        })
    
    return examples

train_data = generate_examples(1, 400) + generate_examples(2, 400) + generate_examples(3, 400) + generate_examples(4, 400)
dataset = Dataset.from_list(train_data)

In [7]:
dataset = dataset.train_test_split(test_size=0.25, seed=42, shuffle=True)
not_preprocessed_val_data = dataset['test']

def map_func(samples):
    return {'text': [template_processing(input_, output) for input_, output in zip(samples['input'], samples['output'])]}

dataset = dataset.map(map_func, batched=True, batch_size=1000)
dataset = dataset.remove_columns(['input', 'output'])
train_dataset, val_dataset = dataset['train'], dataset['test']

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_type=torch.float16,
    bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.5,
    bias='none',
    task_type='CAUSAL_LM'
)

args = TrainingArguments(
    optim='paged_adamw_8bit',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    output_dir='./result',
    report_to='none',
    learning_rate=1e-4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=20,
    eval_steps=20,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_mean_token_accuracy',
    greater_is_better=True,
    num_train_epochs=3,
    fp16=torch.cuda.is_available()
)
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_config,
    args=args
)
trainer.train()
trainer.save_model('./qa_model')

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/1200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1200 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1200 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
20,1.510000,1.278200,1.122997,55761.000000,0.734506
40,0.928600,0.629186,0.723739,110884.000000,0.843044
60,0.509000,0.432846,0.456007,164570.000000,0.882273
80,0.411300,0.397084,0.407156,217180.000000,0.885773
100,0.388900,0.378843,0.387258,271401.000000,0.889123
120,0.378000,0.368159,0.386546,326116.000000,0.891587
140,0.368900,0.360821,0.374802,380313.000000,0.892844
160,0.360200,0.356025,0.371527,435225.000000,0.894318
180,0.354700,0.353490,0.362546,489767.000000,0.894925
200,0.350900,0.352284,0.361779,544387.000000,0.895307


In [9]:
def generate_response(model, tokenizer, prompts):
    inputs = tokenizer(
        prompts, 
        return_tensors='pt', 
        padding=True, 
        truncation=True, 
        max_length=512
    ).to(model.device)
    
    # Переносим на устройство модели
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():  # добавь это
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
        )
    
    # Убираем промпт
    results = []
    input_length = inputs['input_ids'].shape[1]
    for output in outputs:
        generated_ids = output[input_length:]
        text = tokenizer.decode(generated_ids, skip_special_tokens=True)
        results.append(text)
    
    return results

rouge = evaluate.load('rouge')
def evaluate_model(model, tokenizer, val_data, batch_size=8):
    predictions = []

    for i in tqdm(range(0, len(val_data), batch_size)):
        batch = [val_data[j] for j in range(i, min(i+batch_size, len(val_data)))]
        batch_predictions = generate_response(model, tokenizer, [template_processing(item) for item in batch])
        predictions.extend(batch_predictions)

    references = [item['output'] for item in val_data]

    return rouge.compute(predictions=predictions, references=references)

In [15]:
# trainer.model.eval()
generate_response(trainer.model.half(), tokenizer, [template_processing('Маша купила 5 машин за 2500 рублей и 3 кактуса за 4 рубля')])

['assistant\n{"покупки": [{"товар": "машины", "количество": 5, "цена": 2500}, {"товар": "кактуса", "количество": 3, "цена": 4}], "итого": 2504}']

In [13]:
scores = evaluate_model(trainer.model.half(), tokenizer, not_preprocessed_val_data.select(range(30)), batch_size=4)
print(scores)

# здесь метрики плохие, так как они плохо подходят для таких структурированных данных, как json, поэтому надо выбрать другие метрики
# по примеру выше видно, что модель справляется нормально

  0%|          | 0/8 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

{'rouge1': 0.20922677063027942, 'rouge2': 0.20759545923632614, 'rougeL': 0.21026640675763486, 'rougeLsum': 0.21273554256010396}
